# Query Shared MotherDuck Database

## Overview

This notebook demonstrates how to take a shared MotherDuck database and query it using python.

In [35]:
import os
import tomllib
import duckdb

# Load the TOML file
with open("config.toml", "rb") as file:
    config = tomllib.load(file)

# Extract the MD_SERVICE_TOKEN from the loaded config
md_service_token = config['mother-duck']['MD_SERVICE_TOKEN']


con = duckdb.connect(f'md:?motherduck_token={md_service_token}') 

In [36]:
try:
    con.sql("ATTACH 'md:_share/voc_omop54/dd02ef6b-06c3-4292-b957-b50baadecc2e' AS voc_omop54_shared")
except Exception as e:
    if "already attached" in str(e):
        print("Already attached")
    else:
        # Handle other exceptions if needed
        print("An error occurred:", e)

Already attached


In [37]:
con.sql("SHOW DATABASES").show()

┌───────────────────┐
│   database_name   │
│      varchar      │
├───────────────────┤
│ my_db             │
│ voc_omop54        │
│ voc_omop54_shared │
└───────────────────┘



In [51]:
import polars as pl

con.sql("USE voc_omop54_shared")
df = con.sql("SELECT * FROM concept").pl()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [52]:
df

concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
i32,str,str,str,str,str,str,date,date,str
45053734,"""Etodolac 300 M…","""Drug""","""NDC""","""11-digit NDC""",null,"""54569326401""",2007-06-01,2011-01-01,"""D"""
45053735,"""Cephalexin 500…","""Drug""","""NDC""","""11-digit NDC""",null,"""54569332406""",2007-06-01,2012-12-01,"""D"""
45053736,"""Amoxicillin 50…","""Drug""","""NDC""","""11-digit NDC""",null,"""54569333501""",2007-06-01,2012-12-01,"""D"""
45053737,"""12 HR Hyoscyam…","""Drug""","""NDC""","""11-digit NDC""",null,"""54569349600""",2011-01-01,2013-07-01,"""D"""
45053738,"""Chloroprocaine…","""Drug""","""NDC""","""11-digit NDC""",null,"""54569350000""",2010-06-01,2013-07-01,"""D"""
…,…,…,…,…,…,…,…,…,…
45053729,"""Thiamine 100 M…","""Drug""","""NDC""","""11-digit NDC""",null,"""54569288001""",2012-08-01,2017-06-01,"""D"""
45053730,"""Lidocaine Hydr…","""Drug""","""NDC""","""11-digit NDC""",null,"""54569294300""",2010-10-01,2013-07-01,"""D"""
45053731,"""Sulfamethoxazo…","""Drug""","""NDC""","""11-digit NDC""",null,"""54569298604""",2007-06-01,2013-07-01,"""D"""
